In [1]:
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, Float, String, DateTime

# hide ipykernel warnings 
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', 120)

In [2]:
# Connect to database
engine = create_engine(f"mysql+mysqlconnector://{name}:{password}@{host}:3306/{db}", echo=True)
connection = engine.connect()

2021-08-02 23:12:28,582 INFO sqlalchemy.engine.Engine SHOW VARIABLES LIKE 'sql_mode'
2021-08-02 23:12:28,584 INFO sqlalchemy.engine.Engine [raw sql] {}
2021-08-02 23:12:28,592 INFO sqlalchemy.engine.Engine SHOW VARIABLES LIKE 'lower_case_table_names'
2021-08-02 23:12:28,594 INFO sqlalchemy.engine.Engine [generated in 0.00368s] {}
2021-08-02 23:12:28,605 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2021-08-02 23:12:28,606 INFO sqlalchemy.engine.Engine [raw sql] {}


In [3]:
# Read in list of most common subroutes
with open("most_common_subroutes.txt") as file:
    most_common_subroutes = file.readlines()
most_common_subroutes = [line.strip() for line in most_common_subroutes] 

In [4]:
print(most_common_subroutes)

['102_8', '102_9', '104_15', '104_16', '111_7', '111_8', '114_5', '114_6', '116_1', '116_3', '118_4', '11_40', '11_42', '120_7', '120_9', '122_18', '122_20', '123_34', '123_36', '130_10', '130_11', '13_60', '13_67', '140_19', '140_21', '142_12', '142_13', '145_102', '145_105', '14C_17', '14C_18', '14_15', '14_16', '150_8', '150_9', '151_15', '151_17', '15A_83', '15A_84', '15B_60', '15B_61', '15D_62', '15D_63', '15_16', '15_17', '161_50', '161_51', '16C_28', '16C_29', '16D_30', '16_20', '16_24', '17A_15', '17A_17', '17_10', '17_15', '184_28', '184_29', '185_53', '185_56', '18_3', '18_4', '1_37', '1_40', '220_10', '220_12', '236_10', '236_9', '238_11', '238_15', '239_26', '239_28', '25A_270', '25A_273', '25B_271', '25B_274', '25D_275', '25D_277', '25X_10', '25X_11', '25_269', '25_272', '26_28', '26_29', '270_42', '270_44', '27A_4', '27A_5', '27B_23', '27B_34', '27X_42', '27X_43', '27_17', '27_19', '29A_14', '29A_15', '31A_25', '31A_26', '31B_44', '31B_46', '31D_50', '31D_51', '31_15', '3

In [5]:
trips = pd.read_csv("trips_to_match_leavetimes.csv")

In [6]:
trips.head()

,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,JOURNEYTIME
0,07-FEB-18 00:00:00,6253917,102,102_8,1,87253,84600,86999,84588,2411
1,07-FEB-18 00:00:00,6250433,102,102_8,1,29588,26100,30330,26175,4155
2,07-FEB-18 00:00:00,6253905,102,102_8,1,31737,28200,32979,28292,4687
3,07-FEB-18 00:00:00,6260556,102,102_8,1,33237,29700,33432,29770,3662
4,07-FEB-18 00:00:00,6259508,102,102_8,1,34437,30900,34743,30893,3850


In [7]:
trips.dtypes

DAYOFSERVICE       object
TRIPID              int64
LINEID             object
ROUTEID            object
DIRECTION           int64
PLANNEDTIME_ARR     int64
PLANNEDTIME_DEP     int64
ACTUALTIME_ARR      int64
ACTUALTIME_DEP      int64
JOURNEYTIME         int64
dtype: object

In [8]:
trips["LINEID"] = trips["LINEID"].astype(str)

In [9]:
print(trips.LINEID.unique().tolist())

['102', '104', '111', '114', '116', '118', '11', '120', '122', '123', '130', '13', '140', '142', '145', '14C', '14', '150', '151', '15A', '15B', '15D', '15', '161', '16C', '16D', '16', '17A', '17', '184', '185', '18', '1', '220', '236', '238', '239', '25A', '25B', '25D', '25X', '25', '26', '270', '27A', '27B', '27X', '27', '29A', '31A', '31B', '31D', '31', '32X', '32', '33A', '33B', '33D', '33E', '33X', '33', '37', '38A', '38B', '38D', '38', '39A', '39X', '39', '40B', '40D', '40E', '40', '41A', '41B', '41C', '41D', '41X', '41', '42D', '42', '43', '44B', '44', '45A', '46A', '46E', '47', '49', '4', '51D', '51X', '53', '54A', '56A', '59', '61', '63', '65B', '65', '66A', '66B', '66X', '66', '67X', '67', '68A', '68X', '68', '69X', '69', '70D', '70', '75', '76A', '76', '77A', '77X', '79A', '79', '7A', '7B', '7D', '7', '83A', '83', '84A', '84X', '84', '9']


In [10]:
# Initiate dataframe to hold stop proportions
proportions_df = pd.DataFrame(columns=["LINEID", "ROUTEID", "DIRECTION", "PROGRNUMBER", "STOPPOINTID", "STOP_PERCENT"])

In [11]:
# Drop table stop_proportions
drop_query = sqlalchemy.text("DROP TABLE stop_proportions")
# connection.execution_options(autocommit=True).execute(drop_query)

In [12]:
# Empty table stop_proportions of all data (noticed possible mistake in proportion calculation)
truncate_query = sqlalchemy.text("TRUNCATE TABLE stop_proportions")
# connection.execution_options(autocommit=True).execute(truncate_query)

In [13]:
# Calculate proportions for stops on most common subroutes and add to dataframe
for route in most_common_subroutes:
    line = str(route.split("_")[0])
    
    current = pd.read_csv("stoptimes/" + line + "_stops.csv")
    merged = current.merge(trips, how="left", on=["DAYOFSERVICE", "TRIPID"])
    
    route_subset = merged.loc[merged["ROUTEID"] == route]
    route_subset.dropna(inplace=True)
    route_subset["TIME_TO_STOP"] = route_subset["ACTUALARR_STOP"] - route_subset["ACTUALTIME_DEP"]
    
    direction = route_subset.iloc[0]["DIRECTION"]
    
    removal_indices = route_subset[route_subset["TIME_TO_STOP"] < 0].index
    route_subset.drop(removal_indices, inplace=True)
    
    route_subset["STOP_PERCENT"] = route_subset["TIME_TO_STOP"] / route_subset["JOURNEYTIME"] * 100
    route_subset.drop(["DAYOFSERVICE", "TRIPID", "ROUTEID", "PLANNEDDEP_STOP", "PLANNEDARR_STOP", "ACTUALARR_STOP", "ACTUALDEP_STOP", "LINEID", "PLANNEDTIME_ARR", "PLANNEDTIME_DEP", "ACTUALTIME_ARR", "ACTUALTIME_DEP", "JOURNEYTIME", "TIME_TO_STOP"], axis = 1, inplace=True)
    
    stop_list = route_subset.STOPPOINTID.unique().tolist()
    
    for stop in stop_list:
        current_stop = route_subset.loc[route_subset["STOPPOINTID"] == stop]
        median_percent = current_stop.STOP_PERCENT.median()
        new_row_proportions_df = {"LINEID":line, "ROUTEID":route, "DIRECTION":direction, "PROGRNUMBER":current_stop.iloc[0]["PROGRNUMBER"], "STOPPOINTID":stop, "STOP_PERCENT":median_percent}
        proportions_df = proportions_df.append(new_row_proportions_df, ignore_index=True)
    print("ROUTE", route, "complete")

ROUTE 102_8 complete
ROUTE 102_9 complete
ROUTE 104_15 complete
ROUTE 104_16 complete
ROUTE 111_7 complete
ROUTE 111_8 complete
ROUTE 114_5 complete
ROUTE 114_6 complete
ROUTE 116_1 complete
ROUTE 116_3 complete
ROUTE 118_4 complete
ROUTE 11_40 complete
ROUTE 11_42 complete
ROUTE 120_7 complete
ROUTE 120_9 complete
ROUTE 122_18 complete
ROUTE 122_20 complete
ROUTE 123_34 complete
ROUTE 123_36 complete
ROUTE 130_10 complete
ROUTE 130_11 complete
ROUTE 13_60 complete
ROUTE 13_67 complete
ROUTE 140_19 complete
ROUTE 140_21 complete
ROUTE 142_12 complete
ROUTE 142_13 complete
ROUTE 145_102 complete
ROUTE 145_105 complete
ROUTE 14C_17 complete
ROUTE 14C_18 complete
ROUTE 14_15 complete
ROUTE 14_16 complete
ROUTE 150_8 complete
ROUTE 150_9 complete
ROUTE 151_15 complete
ROUTE 151_17 complete
ROUTE 15A_83 complete
ROUTE 15A_84 complete
ROUTE 15B_60 complete
ROUTE 15B_61 complete
ROUTE 15D_62 complete
ROUTE 15D_63 complete
ROUTE 15_16 complete
ROUTE 15_17 complete
ROUTE 161_50 complete
ROUTE 1

In [14]:
# Sort dataframe
proportions_df = proportions_df.sort_values(["ROUTEID", "PROGRNUMBER"])

In [15]:
proportions_df.head()

,LINEID,ROUTEID,DIRECTION,PROGRNUMBER,STOPPOINTID,STOP_PERCENT
16,102,102_8,1.0,1.0,4381,0.000000
17,102,102_8,1.0,2.0,935,1.059135
20,102,102_8,1.0,3.0,936,1.554126
21,102,102_8,1.0,4.0,913,4.036384
52,102,102_8,1.0,5.0,914,5.107714


In [16]:
proportions_df.shape

(12789, 6)

In [17]:
proportions_df.isna().sum()

LINEID          0
ROUTEID         0
DIRECTION       0
PROGRNUMBER     0
STOPPOINTID     0
STOP_PERCENT    0
dtype: int64

In [18]:
proportions_df.dtypes

LINEID           object
ROUTEID          object
DIRECTION       float64
PROGRNUMBER     float64
STOPPOINTID      object
STOP_PERCENT    float64
dtype: object

In [19]:
# Change encoding for direction to match GTFS data (0 for OB and 1 for IB)
proportions_df["DIRECTION"] = proportions_df["DIRECTION"].astype("int64")
proportions_df["DIRECTION"] = proportions_df["DIRECTION"] - 1

In [20]:
proportions_df["PROGRNUMBER"] = proportions_df["PROGRNUMBER"].astype("int64")

In [22]:
proportions_df.head()

,LINEID,ROUTEID,DIRECTION,PROGRNUMBER,STOPPOINTID,STOP_PERCENT
16,102,102_8,0,1,4381,0.000000
17,102,102_8,0,2,935,1.059135
20,102,102_8,0,3,936,1.554126
21,102,102_8,0,4,913,4.036384
52,102,102_8,0,5,914,5.107714


In [24]:
# Create function to add contents of dataframe to database
def create_and_fill_table(df):
    meta = MetaData()
    
    table = Table(
        "stop_proportions", meta, 
        Column("LINEID", String(12)),
        Column("ROUTEID", String(12)),
        Column("DIRECTION", Integer),
        Column("PROGRNUMBER", Integer),
        Column("STOPPOINTID", Integer),
        Column("STOP_PERCENT", Float)
    )
    
    meta.create_all(engine)
    df.to_sql("stop_proportions", con=engine, if_exists="append", index=False)

In [25]:
create_and_fill_table(proportions_df)

2021-08-02 23:24:22,625 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-08-02 23:24:22,631 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %(table_schema)s AND table_name = %(table_name)s
2021-08-02 23:24:22,633 INFO sqlalchemy.engine.Engine [generated in 0.00348s] {'table_schema': 'dubbusdb', 'table_name': 'stop_proportions'}
2021-08-02 23:24:22,641 INFO sqlalchemy.engine.Engine 
CREATE TABLE stop_proportions (
	`LINEID` VARCHAR(12), 
	`ROUTEID` VARCHAR(12), 
	`DIRECTION` INTEGER, 
	`PROGRNUMBER` INTEGER, 
	`STOPPOINTID` INTEGER, 
	`STOP_PERCENT` FLOAT
)


2021-08-02 23:24:22,643 INFO sqlalchemy.engine.Engine [no key 0.00315s] {}
2021-08-02 23:24:22,679 INFO sqlalchemy.engine.Engine COMMIT
2021-08-02 23:24:22,698 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %(table_schema)s AND table_name = %(table_name)s
2021-08-02 23:24:22,700 INFO sqlalchemy.engine.Engine [cached since 0.07084